In [1]:
import pandas as pd
import sqlite3 # 导入 Python 自带的 SQL 数据库库

df_clean = pd.read_csv('netflix_titles_cleaned.csv')

df_clean.head(5)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",Unknown,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,Unknown,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [14]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8793 entries, 0 to 8792
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8793 non-null   object
 1   type          8793 non-null   object
 2   title         8793 non-null   object
 3   director      6172 non-null   object
 4   cast          7968 non-null   object
 5   country       8793 non-null   object
 6   date_added    8793 non-null   object
 7   release_year  8793 non-null   int64 
 8   rating        8793 non-null   object
 9   duration      8790 non-null   object
 10  listed_in     8793 non-null   object
 11  description   8793 non-null   object
dtypes: int64(1), object(11)
memory usage: 824.5+ KB


In [2]:
#     创建一个到“内存”数据库的连接
#    :memory: 是一个特殊的名字，代表数据库只存在于 RAM 中，关闭程序就消失
conn = sqlite3.connect(':memory:')   

In [3]:
#    把我们的 DataFrame 数据“写入”到这个数据库中
#    我们给这个 SQL 表起名叫 "netflix"
df_clean.to_sql('netflix',conn,index=False)
print('数据库已经载入成功！')

#conn 就是你通往数据库的“管道”。现在，所有数据都在 SQL 数据库的 netflix 表里了。

数据库已经载入成功！


In [4]:
#    SQL的五大指令
#    SQl用的是查询语句（Query）

#    在python中需要用pd.read_sql_query调用

In [5]:
# 指令1&2  Select&From

# SQL 查询语句
query1 = """
    SELECT * FROM netflix 
    LIMIT 5;
"""
### *意为着 ‘*’选择了所有列

query2 = '''
    SElECT show_id,type
    FROM netflix
    Limit 5;
'''

# 在 Pandas 中执行 SQL
df_result1 = pd.read_sql_query(query1, conn)

print("--- 1. SELECT * FROM (等同于 df.head(5)) ---")
print(df_result1)

--- 1. SELECT * FROM (等同于 df.head(5)) ---
  show_id     type                  title         director  \
0      s1    Movie   Dick Johnson Is Dead  Kirsten Johnson   
1      s2  TV Show          Blood & Water             None   
2      s3  TV Show              Ganglands  Julien Leclercq   
3      s4  TV Show  Jailbirds New Orleans             None   
4      s5  TV Show           Kota Factory             None   

                                                cast        country  \
0                                               None  United States   
1  Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...   South Africa   
2  Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...        Unknown   
3                                               None        Unknown   
4  Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...          India   

           date_added  release_year rating   duration  \
0  September 25, 2021          2020  PG-13     90 min   
1  September 24, 2021          2021  TV-MA  

In [6]:
df_result2 = pd.read_sql_query(query2,conn)

print(df_result2)

  show_id     type
0      s1    Movie
1      s2  TV Show
2      s3  TV Show
3      s4  TV Show
4      s5  TV Show


In [12]:
###   指令3 通过where进行筛选特定条件


query3 = '''
    Select show_id,type 
    From netflix
    Where country = 'United States' and release_year > 2000
    Limit 5
'''
print("\n--- 2. WHERE 筛选 (等同于 df[...]) ---")
result3 = pd.read_sql_query(query3,conn)
print(result3)



--- 2. WHERE 筛选 (等同于 df[...]) ---
  show_id     type
0      s1    Movie
1     s10    Movie
2     s16  TV Show
3     s28    Movie
4     s29    Movie


In [18]:
### 指令 4&5 排序(Order by)和限制(Limit)

#ORDER BY：按某一列排序。 #这完全等同于 Pandas 的 .sort_values()。

#DESC：代表“降序”(Descending)。(ORDER BY默认是 ASC 升序)。

# SQL 查询语句：选出最新的5部电影
query4 = """
    SELECT title, release_year
    FROM netflix
    WHERE type = 'Movie' AND country = 'United States'
    ORDER BY release_year  DESC
    LIMIT 5;
"""

# 在 Pandas 中执行 SQL
result4 = pd.read_sql_query(query4, conn)

print("\n--- 3. ORDER BY 排序 (等同于 .sort_values().head()) ---")
print(result4)


--- 3. ORDER BY 排序 (等同于 .sort_values().head()) ---
                    title  release_year
0            The Starling          2021
1                    Kate          2021
2  Untold: Breaking Point          2021
3           The Water Man          2021
4              Sweet Girl          2021
